In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
train = pd.read_csv("emnist-balanced-train.csv",delimiter = ',')
test = pd.read_csv("emnist-balanced-test.csv", delimiter = ',')

In [3]:
HEIGHT = 28
WIDTH = 28

In [4]:
# Split x and y
train_x = train.iloc[:,1:]
train_y = train.iloc[:,0]
del train

test_x = test.iloc[:,1:]
test_y = test.iloc[:,0]
del test

In [5]:
def rotate(image):
    image = image.reshape([HEIGHT, WIDTH])
    image = np.fliplr(image)
    image = np.rot90(image)
    return image

In [6]:
# Flip and rotate image
train_x = np.asarray(train_x)
train_x = np.apply_along_axis(rotate, 1, train_x)
test_x = np.asarray(test_x)
test_x = np.apply_along_axis(rotate, 1, test_x)

In [7]:
# Normalise
train_x = train_x.astype('float32')
train_x /= 255
test_x = test_x.astype('float32')
test_x /= 255

In [8]:
# number of classes
num_classes = train_y.nunique()

In [9]:
# One hot encoding
train_y = np_utils.to_categorical(train_y, num_classes)
test_y = np_utils.to_categorical(test_y, num_classes)

In [10]:
# partition to train and val
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size= 0.10, random_state=7)

In [11]:
# Reshape image for CNN
train_x = train_x.reshape(-1, HEIGHT, WIDTH, 1)
test_x = test_x.reshape(-1, HEIGHT, WIDTH, 1)

In [12]:
callbacks = [
    EarlyStopping(
        monitor='val_acc', 
        patience=36,
        mode='max',
        verbose=1),
    ModelCheckpoint('cnn_model_training1.h5',
        monitor='val_acc', 
        save_best_only=True, 
        mode='max',
        verbose=0)
]

# input_shape = (28, 28, 1)
filter_pixel = 3
droprate = 0.25 #can tuning

model = Sequential()

#convolution 1st layer
model.add(Conv2D(64, kernel_size=(filter_pixel, filter_pixel), padding='same', activation='relu', input_shape=(28,28,1)))
model.add(BatchNormalization())
model.add(Dropout(droprate))

#convolution 2nd layer
model.add(Conv2D(64, kernel_size=(filter_pixel, filter_pixel), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Dropout(droprate))

#convolution 3rd layer
model.add(Conv2D(64, kernel_size=(filter_pixel, filter_pixel), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Dropout(droprate))

#Fully connected 1st layer
model.add(Flatten())
model.add(Dense(128,use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(droprate))

#Fully connected final layer
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=10, batch_size=512, verbose=1, validation_data=(val_x, val_y), shuffle=True, callbacks=callbacks)

Epoch 1/10
  5/199 [..............................] - ETA: 7:46 - loss: 3.8301 - accuracy: 0.0754

KeyboardInterrupt: 

In [ ]:
# plot accuracy and loss
def plotgraph(epochs, acc, val_acc, type):
    plt.plot(epochs, acc, 'b')
    plt.plot(epochs, val_acc, 'r')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()
    if (type == "Accuracy"):
        plt.title('Model accuracy')
        plt.ylabel('Accuracy')
    else:
        plt.title('Model loss')
        plt.ylabel('Loss')

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1,len(acc)+1)

In [ ]:
plotgraph(epochs, acc, val_acc, "Accuracy")

In [ ]:
# loss curve
plotgraph(epochs, loss, val_loss, "Loss")

In [ ]:
score = model.evaluate(test_x, test_y, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])